# CS 4700 - Homework #2
#### Due Date: Wednesday, 11/14 @ 6pm on CMS

---
### Section 1: Introduction
In this assignment, you will be implementing a decision tree learner in Python 3. Before we can start, let us import some functions that we will need. Please ensure the file *utils.py* is in the same directory as your Jupyter Notebook.

In [2]:
from utils import *
from math import log
import operator

Recall that the decision tree learner will take in, as input, a dataset containing examples. These examples each contain a sequence of attributes. Given some target attribute, the learner will create a decision tree that will (hopefully) be shallow and, for each example in the dataset, output the correct value for the target attribute. Before we begin to code the learner, we will need to first define what exactly is a decision tree. First, let's begin by defining the leaf node of a decision tree.

In [3]:
"""
A leaf node of a decision tree which holds just a result.
"""
class DTLeaf:
    def __init__(self, result):
        self.result = result

    def __call__(self, example):
        return self.result

    def display(self, indent = 0):
        print('Result =', self.result)

Second, we define the internal (meaning non-leaf) node of the decision tree.

In [4]:
"""
An internal node of a decision tree which tests an attribute, along with a
dictionary of branches, one for each of the attribute's values.
"""
class DTInternal:
    """
    Initialize by saying what attribute this node tests.
    """
    def __init__(self, attr, attrName = None, defaultChild = None,
                 branches = None):
        self.attr = attr
        self.attrName = attrName or attr
        self.defaultChild = defaultChild
        self.branches = branches or {}


    """
    Given an example, classify it using the attribute and the branches.
    """
    def __call__(self, example):
        attrVal = example[self.attr]
        if attrVal in self.branches:
            return self.branches[attrVal](example)
        else:
            return self.defaultChild(example)


    """
    Add a branch.
    """
    def add(self, val, subtree):
        self.branches[val] = subtree


    """
    Prints out the decision tree starting at this internal node using recursion.
    """
    def display(self, indent = 1):
        print ('Testing attribute:', self.attrName)
        for (val, subtree) in self.branches.items():
            print(' ' * 4 * indent, self.attrName, '=', val, '==>', end = ' ')
            subtree.display(indent + 1)
        print()

Below are 4 functions that will prove to be useful in your implementation of the decision tree learner. Take some time to go over what each function is doing and familiarize yourself with how to use them. This will make the assignment easier.

In [5]:
"""
Multiply each number by a constant such that the sum is 1.0
"""
def normalize(dist):
    if isinstance(dist, dict):
        total = sum(dist.values())
        for key in dist:
            dist[key] = dist[key] / total
            assert 0 <= dist[key] <= 1, 'Probabilities must be between 0 and 1.'
        return dist
    total = sum(dist)
    return [(n / total) for n in dist]


"""
Return a copy of the input with all occurrences of item removed.
"""
def removeAll(item, seq):
    if isinstance(seq, str):
        return seq.replace(item, '')
    else:
        return [x for x in seq if x != item]


"""
Randomly shuffle a copy of the input.
"""
def shuffled(seq):
    items = list(seq)
    random.shuffle(items)
    return items


"""
Return an element with the highest value according to key, which is a function.
We break ties by first shuffling the input.
"""
def argmaxRandomTie(seq, key):
    return max(shuffled(seq), key = key)

With a decision tree now defined, we can begin to code the learner.

---
### Section 2: The First Half of the Learner
We will be modeling our code closely to the pseudo-code from the textbook. Your first task is to implement the function *__pluralityVal__*. Recall that, given examples, we define the plurality value as the value of the target attribute that appears most often. As a hint, you will likely need to use both *__count__* (defined below) as well as *__argmaxRandomTie__*. Note, to make *__pluralityVal__* work easier with the learner, it should return a leaf node containing the plurality value, rather than just the value itself. We have also provided a small test to check if your function is returning the correct value.

In [6]:
"""
Count the number of examples that have example[attr] = val.
"""
def count(attr, val, examples):
    return sum(e[attr] == val for e in examples)


"""
Purpose: A default value needs to be returned when there are no examples for a combination of attribute values. This is achieved
         by returning the Plurality Value of all the examples that were used to construct the node's parent.
Inputs: examples = te , values = tv , target = tt
Outputs: Leaf Node of Decision Tree which holds the element with the highest value according to the function key(lambda: key)
Logic: We first compute the function that needs to be passed as key to the argmaxRandomTie. This is done by using the count
       function and passing tv[tt] to retrieve the count for each element in tv[tt] i.e [0,1,2]. For each of these values, the
       argmaxRandomTie determines the count values and returns the maximum(count(tv[tt])). This is then created as an object of
       DTLeaf and returned.
"""
def pluralityVal(examples, values, target):
    
    newLeaf = argmaxRandomTie(values[target],lambda key: count(target,values,examples))
    return DTLeaf(newLeaf)
    


# Test for pluralityVal
te = [[3,2,1], [1,0,1], [4,3,2], [3,0,0], [5,2,1]]
tv = [[1,2,3,4,5], [0,1,2,3], [0,1,2]]
tt = 2
assert pluralityVal(te, tv, tt).result == 1, 'Failed pluralityVal test'

Next, we need to determine if all the examples have the same classification. We will do this in the function *__allSameClass__*, which should simply return a Boolean value. Once again, a couple of tests have been added to ensure you are returning the correct output.

In [7]:
"""
Purpose: We need a method to check if the the examples are split according to their Goal States so that we need not
         explore that branch further. allSameClass determines if the all examples at the child after splitting the parent on some 
         attributes fall in either of the two goal states.
Inputs: examples = te , target = tt
Outputs: Return False if te[tt] is not same for all the elements in te. Else return True
Logic: We take the value in the first element of te at index tt and assign it to indvalue. Later loop through all the elements
       present in te comparing the te[tt] value to indvalue to determine the output.
Tricks: Take the value at index [target] in examples and compare it to the rest of the examples at the same index
"""
def allSameClass(examples, target):
    indvalue = examples[0][target]
    
    for example in examples:
        if(example[target] != indvalue):
            return False
        else:
            continue
    
    return True

# Tests for allSameClass
te = [[3,2,1], [1,0,1], [4,3,2], [3,0,0], [5,2,1]]
tt = 1
assert not allSameClass(te, tt), 'Failed allSameClass test #1'
te = [[3,0,1], [1,0,1], [4,0,2], [3,0,0], [5,0,1]]
assert allSameClass(te, tt), 'Failed allSameClass test #2'

At this point, we have completed enough functions for the first half of the decision tree learner. We will now focus on the second half.

---
### Section 3: The Second Half of the Learner
We need to some way to determine the most important attribute in our dataset that we should split next on for our tree. There are many ways for us to define "importance." One common way to do so is by looking at the information gain that an attribute provides. This gain is defined in terms of the entropy. Recall that entropy can be calculated as:

$$
\begin{aligned}
H(V) &= - \sum_{k} P(v_k)\ log_2 P(v_k)
\end{aligned}
$$

Note that the above definition relies on probabilities. Can you assume that the attribute values being passed in are probabilities? Consider the answer to this in your implementation of this equation in the function *__entropy__*. If correct, you should be able to pass the 2 tests.

In [8]:
"""
Purpose: Entropy is required to calculate the uncertainty of a random variable. Here this random variable are the set of values 
         which it can take. In Binary case,(B) the set of values is either true(1) or false(0). But in Multi Value case(H), this
         set can contain any admissible value.
Inputs: values = tv
Outputs: entropy of the values which is calculated using the formula −∑kP(vk)log2P(vk)
Logic: We first calculate the sum of the values. Iterate over each value in values and divide it by the sum to get the probability 
       of that value and append it into an array. Now we have a list of probabilities for each value in values. With this, we check
       if that element is 0 since log(0) is undefined and use the formula to calculate the entropy.
Tricks: Each element of probability is converted to float for better accuracy
"""
def entropy(values):
    probarray = []
    entropy = []
    summ = sum(values)
    if summ == 0:
        return 0
    for i in values:
        probarray.append(i/summ)
    
    for i in probarray:
        if i != 0:
            ifloat = float(i)
            entropy.append(ifloat*math.log2(ifloat))
        
    return(-1*sum(entropy))
    

# Tests for entropy
tv = [13, 5, 2, 20, 4, 10, 4]
assert abs(entropy(tv) - 2.4549947941466774) <= 1e-3, 'Failed entropy test #1'
tv = [0, 0, 5, 0, 0, 0, 0]
assert entropy(tv) == 0.0, 'Failed entropy test #2'

With the entropy now being calculated, we are able to determine the information gain of an attribute. The calculation for this is as follows:

$$
\begin{aligned}
B(q) &= -[q\ log_2 q + (1 - q)\ log_2 (1 - q)] \\
Remainder(A) &= \sum_{k = 1}^{d} \frac{p_k + n_k}{p + n}\ B(\frac{p_k}{p_k + n_k}) \\
InfoGain(A) &= B(\frac{p}{p + n}) - Remainder(A)
\end{aligned}
$$

As a reminder, \\(q\\) is the probability of a Boolean random variable being true, \\(p\\) is the number of positive examples in the training set, \\(n\\) is the number of negative examples, while their subscripted versions are analogous but for the different values of attribute \\(A\\). You will need to implement the above calculations in the function *__infoGain__*. You might find it useful to use *__splitBy__* (defined below), since calculating \\(Remainder\\) requires us to split an attribute \\(A\\) by its \\(d\\) distinct values.

In [9]:
"""
Return a list of (value, examples) pairs for each val of attr.
"""
def splitBy(attr, examples, values):
    return [(v, [e for e in examples if e[attr] == v]) for v in values[attr]]


"""
Purpose: InfoGain is used to choose the attribute which provides the maximum gain. In other words, it provides a formal measure
         to evaluate the attributes such that the children produced after the split would most likely fall in same Goal States.
        The children may still contain a mixed set of examples which is then split again 
         on some attribute by using InfoGain. The attribute that gives the maximum Gain is the attribute on which the tree is 
         split. The Remainder function yields the expected etnropy remaining after testing the attribute evaluated. 
Inputs: attributes = ta , examples = te , values = tv , target = tt
Outputs: The InfoGain from the attribute test on attr to yield the expected reduction in entropy.
Logic: 1)We first use the count function to retrieve the number of times each value is repeated at target location in examples.
       Since the numbers at the target location in examples is not Binary(0,1,2), we use the entropy function from above to 
       calculate the uncertainity of these values. Next we use the splitBy function to retrieve the value and the examples that
       contain the value at the index attribute in examples. This gives us:
       x = [(1, [[1, 0, 1]]), (2, []), (3, [[3, 2, 1], [3, 0, 0]]), (4, [[4, 3, 2]]), (5, [[5, 2, 1]])]. In other words, we search
       through examples for each value where examples[attribute] = value and map them. Hence 1 is mapped to 1,0,1 since the first
       index has the value 1. No examples start with 2. 3,2,1 and 3,0,0 start with 3 and hence mapped to 3 and so on.
       2) Next we calculate the sum of the lengths of the second parameter which gives us the total example size(p+n).
       3) Next for each term in x we need to decide the positive and negative states for that example. For this, we use the count
       and the entropy function from above. ∑(pk+nk)/(p+n)
       4) Now we calculate each term of the remainder by multiplying two terms,First Term and Second Term. The First Term is 
       nothing but the length of the length of the second column in x divided by the term calculated in 2)(H(p/p+n)). 
       The Second Term is entropy as calculated by 3). All these terms are added up to give the remainder.
       5)Subtract 5) from 1) to get the info gain on the attribute.  
"""
def infoGain(attr, examples, values, target):
    cnt = []
    for val in values[target]:
        cnt.append(count(target,val,examples)) 
    
    newentropy = entropy(cnt) #B(p/(p+n))
    
    x = splitBy(attr,examples,values)
    totallength = 0
    for i,j in x:
        totallength += len(j)
    
    innercnt = []
    summultiply = 0
    for i,j in x:
        firstterm = len(j)/totallength

        for val in values[target]:
            innercnt.append(count(target,val,j))
        
        
        secondentropy = entropy(innercnt)
        summultiply += firstterm*secondentropy
        innercnt = []
        
    return (newentropy - summultiply)
    

# Tests for infoGain
te = [[3,2,1], [1,0,1], [4,3,2], [3,0,0], [5,2,1]]
tv = [[1,2,3,4,5], [0,1,2,3], [0,1,2]]
tt = 2
ta = 0
assert abs(infoGain(ta, te, tv, tt) - 0.9709505944546687) <= 1e-3, \
       'Failed infoGain test #1'
tt = 1
assert abs(infoGain(ta, te, tv, tt) - 1.121928094887362) <= 1e-3, \
       'Failed infoGain test #2'

The last bit of functionality that our learner needs is to determine which attribute we should choose to split on. To recap, the decision tree learner greedily chooses the attribute that provides the most information gain as the splitting point. Capture this logic in the function *__chooseAttr__*.

In [10]:
"""
Purpose: Now that we know what InfoGain is used for, each attribute is evaluated for its InfoGain and the attribute that gives
         maximum InfoGain is used to split the decision tree.
Inputs: attrs = ta , examples = te , values = tv , target = tt
Outputs: The attribute that will be used to split the decision tree based on the maximum InfoGain.
Logic: For each of the attributes provided in attrs, we evaluate the InfoGain and find the attribute that yields the maximum 
       InfoGain and return that attribute.
Tricks: Start with a maximum as negative infinity and then keep assigning the maximum as and when encountered
"""
def chooseAttr(attrs, examples, values, target):
    maxi = -math.inf
    predictedattr = 0
    for val in attrs:
        x = infoGain(val, examples, values, target)
        if(x > maxi):
            maxi = x
            predictedattr = val

    return predictedattr


# Test for chooseAttr
te = [[6,7,8,7,0], [7,3,4,0,0], [1,0,2,3,3], [8,5,9,1,1], [9,1,5,4,2],
      [9,5,6,8,3], [4,6,0,1,6], [8,5,4,4,2], [0,3,7,2,6], [7,9,3,1,2]]
tv = [[0,1,2,3,4,5,6,7,8,9] for i in range(10)]
tt = 3
ta = [0,1,2,4]
assert chooseAttr(ta, te, tv, tt) == 2, 'Failed chooseAttr test'

We now have all the necessary functions to implement our learner.

---
### Section 4: Putting It All Together
With everything now in place, we should now be able to implement the decision tree learner in full. For easier reference, the pseudo-code from the textbook (fig. 18.5) has been reproduced below. Use this to help guide your implementation in the function *__DTLearner__*.

**function** DECISION-TREE-LEARNING(*examples*, *attributes*, *parent_examples*) **returns** a tree<br/>
&nbsp;&nbsp;**if** *examples* is empty **then return** PLURALITY-VALUE(*parent_examples*)<br/>
&nbsp;&nbsp;**else if** all *examples* have the same classification **then return** the classification<br/>
&nbsp;&nbsp;**else if** *attributes* is empty **then return** PLURALITY-VALUE(*examples*)<br/>
&nbsp;&nbsp;**else**<br/>
&nbsp;&nbsp;&nbsp;&nbsp;\\(A \leftarrow argmax_{a \in attributes}\\) IMPORTANCE(\\(a\\), *examples*)<br/>
&nbsp;&nbsp;&nbsp;&nbsp;*tree* \\(\leftarrow\\) a new decision tree with root test \\(A\\)<br/>
&nbsp;&nbsp;&nbsp;&nbsp;**for each** value \\(v_k\\) of \\(A\\) **do**<br/>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*exs* \\(\leftarrow\\) {\\(e : e \in\\) *examples* **and** *e.A* = \\(v_k\\)}<br/>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;*subtree* \\(\leftarrow\\) DECISION-TREE-LEARNING(*exs*, *attributes* − \\(A\\), *examples*)<br/>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;add a branch to *tree* with label (\\(A\\) = \\(v_k\\)) and subtree *subtree*<br/>
&nbsp;&nbsp;&nbsp;&nbsp;**return** *tree*

In [11]:
"""
Purpose: With all the necessary functions defined, we design a Decision Tree Learner. This DTLearner knows which tree to expand
         based on the attributes(inturn based on InfoGain) such that the leaf nodes contain same Goal States.
Inputs: examples = te, attrs = ta, values = tv , target = tt, attrNames = ta , parentExamples = default value calculated 
        from the plurality classification of all the examples that were used in constructing the node’s parent 
Outputs: A Decision Tree Learner
Logic: 1) if the examples are empty meaning that they are already properly evaluated into corresponding leaf nodes such that
          each leaf node contains examples of same Goal State then return the Plurality of their parents.
       2) if all of the examples are proved to be in the same Goal State, then we just return that Leaf
       3) if the attributes are empty meaning that there is no proper attribute given or to evaluate on, then return the 
          Plurality of the examples.
       4) if none of 1,2,3 hold then we first choose the attribute based on maximum InfoGain to split the decision tree.
          Next we create an internal node of a decision tree which tests an attribute, along with a dictionary of branches, 
          one for each of the attribute's values by using DTInternal. Next we split the DecisionTree based on the attribute chosen
          and create a dictionary of the children. Remove the attribute from the list since it was already evaluated. The Children
          are again subject to the DTLearner to evaluate which attribute and branch need to be expanded further. This recursion
          is performed until all the examples reach their respective Goal States.
          
"""
def DTLearner(examples, attrs, values, target, attrNames, parentExamples = ()):
    if not examples:
        return pluraityVal(parentExamples,values,target)
    elif allSameClass(examples, target):
        return DTLeaf(examples[0][target])
    elif not attrs:
        return pluraityVal(examples,values,target)
    else:
        A = chooseAttr(attrs, examples, values, target)
        tree = DTInternal(A)
        splitby = splitBy(A,examples,values)
        diction = dict(splitby)
        attrs.remove(A)
        for x in values[A]:
            ex = DTLearner(diction[x] , attrs,values,target,attrNames,examples)
            tree.add(x,ex)
        return tree


"""
Test for DTLearner

Output should match:

Testing attribute: 7
     7 = 0 ==> Result = 1
     7 = 1 ==> Result = 0
     7 = 2 ==> Testing attribute: 11
         11 = 0 ==> Result = 0
         11 = 1 ==> Result = 2
         11 = 2 ==> Result = 1
"""
te = [[1,1,1,1,1,1,2,2,2,0,2,0,0,1,2,0,0,2,2,0],
      [2,0,1,2,1,1,1,2,2,2,2,1,0,2,2,0,2,2,2,2],
      [1,2,1,1,1,2,2,2,0,1,2,2,2,2,2,1,2,0,2,1],
      [2,2,2,1,0,1,2,2,2,0,2,1,0,1,1,1,0,2,0,2],
      [0,1,1,0,0,0,2,0,1,0,1,2,2,2,2,0,0,0,1,1],
      [0,2,0,0,0,1,0,1,0,2,1,1,2,0,2,2,0,2,0,0],
      [0,0,1,2,0,0,0,0,1,2,0,0,2,0,0,0,0,2,1,1],
      [0,2,2,1,1,0,0,2,2,0,2,1,1,0,0,2,0,2,1,2],
      [1,0,2,0,1,2,2,1,0,1,0,2,1,2,0,0,1,1,2,0],
      [0,2,0,1,2,1,1,1,1,0,1,2,2,0,1,2,1,0,0,0]]
tv = [[j for j in range(3)] for i in range(20)]
tt = 19
ta = [i for i in range(19)]
tn = ta
DTLearner(te, ta, tv, tt, tn).display()

Testing attribute: 7
     7 = 0 ==> Result = 1
     7 = 1 ==> Result = 0
     7 = 2 ==> Testing attribute: 11
         11 = 0 ==> Result = 0
         11 = 1 ==> Result = 2
         11 = 2 ==> Result = 1




If all the above is working, we should now be able to train, display, and test our decision tree! The code block below will load in the restaurant data from the textbook, use it as training data for our learner, output the learned decision tree, and then use that to try to classify a test point. To help with this process, we use the *__DTLCaller__* to extract the relevant information from our dataset and pass that into the learner. Please ensure you have the *restaurant.csv* in the same directory as your Jupyter Notebook.

In [12]:
"""
Front-end caller for the decision tree learner. Given an input dataset, it will
extract the necessary information and pass it in to the learner.
"""
def DTLCaller(dataset):
    return DTLearner(dataset.examples, dataset.inputs, dataset.values,
                     dataset.target, dataset.attrNames)


# Load the restaurant dataset from a CSV file
restaurant = Dataset(name = 'restaurant', target = 'Wait',
                     attrNames = 'Alternate Bar Fri/Sat Hungry Patrons Price ' +
                     'Raining Reservation Type WaitEstimate Wait')


# Feed the dataset into the decision tree learner
dtlRestaurant = DTLCaller(restaurant)


# Display the decision tree that is learned
dtlRestaurant.display()


# Try classifying new test data
print(dtlRestaurant(['Yes','No','No','Yes','Full',
                     '$$','No','No','Italian','0-10']))

Testing attribute: 4
     4 = None ==> Result = No
     4 = Full ==> Testing attribute: 8
         8 = Thai ==> Testing attribute: 2
             2 = Yes ==> Result = Yes
             2 = No ==> Result = No

         8 = Burger ==> Testing attribute: 0
             0 = Yes ==> Result = Yes
             0 = No ==> Result = No

         8 = French ==> Result = Yes
         8 = Italian ==> Result = No

     4 = Some ==> Result = Yes

No


We can also use additional datasets. For example, the code block below uses a zoo dataset to train our learner and find a decision tree for our data. The zoo dataset contains a list of animals, each of which has different features about the animal, along with what kind of animal it is. We use our learned decision tree to determine what kind of animal a dragonfly is. Please ensure you have the *zoo.csv* in the same directory as your Jupyter Notebook.

In [13]:
# Load the zoo dataset from a CSV file
zoo = Dataset(name = 'zoo', target = 'type', exclude = ['name'],
              attrNames = 'name hair feathers eggs milk airborne aquatic ' +
              'predator toothed backbone breathes venomous fins legs tail ' +
              'domestic catsize type')


# Feed the zoo dataset into the decision tree learner
dtlZoo = DTLCaller(zoo)


# Display the decision tree that is learned
dtlZoo.display()


# Try classifying new test data
print(dtlZoo(['dragonfly',0,0,1,0,1,0,1,0,0,1,0,0,6,0,0,0]))

Testing attribute: 13
     13 = 0 ==> Testing attribute: 12
         12 = 0 ==> Testing attribute: 8
             8 = 0 ==> Result = shellfish
             8 = 1 ==> Result = reptile

         12 = 1 ==> Testing attribute: 3
             3 = 0 ==> Result = mammal
             3 = 1 ==> Result = fish


     13 = 2 ==> Testing attribute: 1
         1 = 0 ==> Result = bird
         1 = 1 ==> Result = mammal

     13 = 4 ==> Testing attribute: 4
         4 = 0 ==> Testing attribute: 6
             6 = 0 ==> Result = reptile
             6 = 1 ==> Testing attribute: 9
                 9 = 0 ==> Result = shellfish
                 9 = 1 ==> Result = amphibian


         4 = 1 ==> Result = mammal

     13 = 5 ==> Result = shellfish
     13 = 6 ==> Testing attribute: 10
         10 = 0 ==> Result = shellfish
         10 = 1 ==> Result = insect

     13 = 8 ==> Result = shellfish

insect


---
### Section 5: Submission
You will only be submitting your Jupyter Notebook file, *hwk2.ipynb*. Do not worry about submitting the additional files (*utils.py*, *restaurant.csv*, *zoo.csv*). Furthermore, as a reminder, part of your grade is your documentation. Each of the functions that you implemented as part of this assignment **must** be documented. Explain what the function is doing, its inputs/outputs, the logic behind your implementation, fancy Python ~~hacks~~ tricks, etc. Failure to include detailed documentation will result in a penalty.

Please upload your *hwk2.ipynb* file to CMS by **Wednesday, 11/14 @ 6pm**.